In [23]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib as plt
import random
from google.colab.patches import cv2_imshow
%matplotlib inline

In [4]:
random.seed(777)
torch.manual_seed(777)

In [8]:
training_epochs = 15
batch_size = 100

In [6]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [9]:
data_loader = DataLoader(dataset = mnist_train, batch_size=batch_size, shuffle=True , drop_last= True)

In [11]:
#모델 설계
linear = nn.Linear(784,10,bias=True).to('cuda') #GPU 연산 사용할 경우 cuda

In [12]:
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = torch.optim.SGD(linear.parameters(),lr=0.1)

In [15]:
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for X,Y in data_loader:
    #배치 크기가 100이므로 x는 (100,784) 텐서
    X = X.view(-1,28*28).to('cuda')
    #레이블은 0~9 사이의 정수
    Y = Y.to('cuda')

    optimizer.zero_grad()
    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: ', '%94d' % (epoch +1), 'cost =', '{:9f}'.format(avg_cost))

print('Learning finished')

Epoch:                                                                                               1 cost =  0.359338
Epoch:                                                                                               2 cost =  0.331415
Epoch:                                                                                               3 cost =  0.316436
Epoch:                                                                                               4 cost =  0.307189
Epoch:                                                                                               5 cost =  0.300453
Epoch:                                                                                               6 cost =  0.295007
Epoch:                                                                                               7 cost =  0.290893
Epoch:                                                                                               8 cost =  0.287315
Epoch:                                  

In [ ]:
#model test
with torch.no_grad():   #gradient 계산을 수행하지 않는다.
  X_test = mnist_test.test_data.view(-1,28*28).float().to('cuda')
  Y_test = mnist_test.test_labels.to('cuda')

  prediction = linear(X_test)
  correct_prediction = torch.argmax(prediction , 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())


  #MNIST test data에서 무작위로 하나 뽑아서 예측
  r =random.randint(0,len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to('cuda')
  Y_single_data = mnist_test.test_labels[r:r+1].to('cuda')

  print('Label:',Y_single_data.item())
  single_prediction = linear(X_single_data)
  print('Prediction:', torch.argmax(single_prediction,1).item())

  plt.imshow(mnist_test.test_data[r:r + 1].view(28, 28), cmap='Greys', interpolation='nearest')
  plt.show()